### Feature Extraction

In [1]:
import json
import pandas as pd
import os
from pathlib import Path
#set current working directory to the root 
if not os.getcwd().endswith('ratio-hackathon-23'):
    os.chdir('../../')
print(os.getcwd())

/home/aqvbw/Jena/research/conferences-meetings/RATIO-hackathon-23/ratio-hackathon-23/dockerize


In [67]:
#! ls data

In [68]:
from requests import get
from pandas import DataFrame, read_xml
from pathlib import Path


def download_read_topics(url: str, path: Path) -> DataFrame:
    if not path.exists():
        with path.open("wb") as file:
            file.write(get(url).content)
    return read_xml(path).rename(columns={"number": "qid", "title": "query"}).drop(columns=["description", "narrative"])

topics_task_1 = download_read_topics(
    "https://touche.webis.de/clef23/touche23-data/topics-task1.xml",
    Path("topics_task_1.xml")
)

In [80]:
#topics_task_1[topics_task_1.qid == 1]['query'].values[0]

'Should teachers get tenure?'

In [85]:
from nltk.tokenize import sent_tokenize
import numpy as np

def clean_text(doc):
    sentences = str(doc).split('\n')
    sentences = [sent_tokenize(sent) for sent in sentences if sent]
    sentences = [item for sublist in sentences for item in sublist]
    sentences = [s for s in sentences if len(s) > 10]
    if len(nltk.word_tokenize(" ".join(sentences))) > 0:
        return " ".join(sentences)
    else:
        return np.nan
    
def fetch_original_query(qid):
    return topics_task_1[topics_task_1.qid == qid]['query'].values[0]

500


In [88]:
#df.tail(20)

In [89]:
#define feature list
feature_list = [
"LENGTH_OF_DOCUMENT",
"LENGTH_OF_PARAGRAPHS",
"LENGTH_OF_SENTENCES",
"LENGTH_OF_WORDS",
"STOPWORD_NUM",
"SPECCHARS_NUM",
"NUMERICS_NUM",
"FULLUPPERCASEWORD_NUM",
"UPPERCASEWORD_NUM",
"UPPERCASESENTENCE_NUM",
"VOCABULARY_RICHNESS",
"VOCABULARY_RICHNESS_WITHOUT_STOPS",
"ACADEMIC_WORDS",
"PROFANITY_WORDS",
"TOPIC_KEYWORDS",
"ARGUMENTATIVE_WORDS",
"ARGUMENT RATIO",
"SUBJECTIVITY",
"LINKS_NUM",
"SENTIMENT",
"NAMEDENTITIES_NUM",
"SPELLINGMISTAKES_NUM",
"READABILITY",
"SENTENCE_TYPE"
]

In [49]:
import string
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.chunk import ne_chunk
from nltk.util import ngrams
import textblob
from textblob import Word
import enchant
from readability import Readability
import requests

In [29]:
# aggregate external sources
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

enchant_dict = enchant.Dict("en_US") # Create an English dictionary object

[nltk_data] Downloading package stopwords to /home/aqvbw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/aqvbw/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/aqvbw/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/aqvbw/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/aqvbw/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aqvbw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/aqvbw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Download

In [30]:
# load local sources
academic_word_list = Path("./code/notebooks/sources/academic_word_list.txt").open('r').read().split('\n')
academic_word_list = [word.strip() for word in academic_word_list if word.strip() != '']

profanity_word_list = Path("./code/notebooks/sources/profanity_word_list.txt").open('r').read().split('\n')
profanity_word_list = [word.strip() for word in profanity_word_list if word.strip() != '']

In [90]:
KEYWORDS_FROM_QUERY = {
    "Are gas prices too high?": ["gas prices", "cost of fuel", "oil prices"],
    "Are social networking sites good for our society?": ["social networking sites", "social media", "online platforms", "society"],
    "Are video games art?": ["video games", "art", "entertainment"],
    "Can alternative energy effectively replace fossil fuels?": ["alternative energy", "renewable energy", "fossil fuels"],
    "Do animals have rights?": ["animal rights", "animal welfare", "animal testing"],
    "Do electronic voting machines improve the voting process?": ["electronic voting machines", "voting process", "voting technology"],
    "Do humans have free will?": ["free will", "determinism", "philosophy"],
    "Do politicians have no right to privacy?": ["politicians", "privacy", "ethics", "public life"],
    "Do standardized tests improve education?": ["standardized tests", "education", "testing", "student performance"],
    "Do violent video games contribute to youth violence?": ["violent video games", "youth violence", "media effects"],
    "Does lowering the federal corporate income tax rate create jobs?": ["corporate income tax", "tax rate", "job creation"],
    "Does poverty cause crime?": ["poverty", "crime", "socioeconomic factors"],
    "How should nuclear waste be stored?": ["nuclear waste", "storage", "radioactive material"],
    "Is a college education worth it?": ["college education", "higher education", "return on investment"],
    "Is a two-state solution an acceptable solution to the Israeli-Palestinian conflict?": ["two-state solution", "Israeli-Palestinian conflict", "peace process"],
    "Is capitalism the best form of economy?": ["capitalism", "economy", "economic systems"],
    "Is cell phone radiation safe?": ["cell phone radiation", "health risks", "radio frequency"],
    "Is chess a sport?": ["chess", "sports", "mind sports"],
    "Is communism better than democracy?": ["communism", "democracy", "political systems"],
    "Is drinking milk healthy for humans?": ["milk", "health", "nutrition"],
    "Is feminism still needed?": ["feminism", "gender equality", "women's rights"],
    "Is genetically modified food unsafe?": ["genetically modified food", "GMOs", "food safety"],
    "Is golf a sport?": ["golf", "sports", "athleticism"],
    "Is hell a real place?": ["hell", "afterlife", "religious beliefs"],
    "Is homework beneficial?": ["homework", "education", "student learning"],
    "Is human activity primarily responsible for global climate change?": ["climate change", "global warming", "human impact"],
    "Is obesity a disease?": ["obesity", "disease", "health"],
    "Is psychology a science?": ["psychology", "science", "behavioral science"],
    "Is sexual orientation determined at birth?": ["sexual orientation", "nature vs nurture", "human development"],
    "Is the earth flat?": ["flat earth", "conspiracy theories", "science"],
    "Is utilitarianism morally acceptable?": ["utilitarianism", "ethics", "moral philosophy"],
    "Is vaping with e-cigarettes safe?": ["e-cigarettes", "vaping", "health risks"],
    "Is wind power the best alternative energy source?": ["wind power", "alternative energy", "renewable energy"],
    "Should Scotland become independent?": ["Scotland", "independence", "referendum", "UK politics"],
    "Should Turkey join the EU?": ["Turkey", "European Union", "EU membership", "international relations"],
    "Should abortion be legal?": ["abortion", "reproductive rights", "women's health"],
    "Should adults have the right to carry a concealed handgun?": ["concealed carry", "guns", "Second Amendment", "gun control"],
    "Should animals be used for scientific or commercial testing?": ["animal testing", "ethical treatment of animals", "scientific research"],
    "Should any vaccines be required for children?": ["vaccines", "childhood immunizations", "public health"],
    "Should birth control pills be available over the counter?": ["birth control", "contraception", "pharmaceuticals"],
    "Should blood donations be financially compensated?": ["blood donations", "compensation", "donor incentives"],
    "Should body cameras be mandatory for police?": ["police", "body cameras", "law enforcement"],
    "Should bottled water be banned?": ["bottled water", "environmental impact", "sustainability"],
    "Should bullfighting be banned?": ["bullfighting", "animal cruelty", "cultural practices"],
    "Should children beauty contests be banned?": ["beauty pageants", "children", "objectification"],
    "Should children have mobile phones?": ["children", "mobile phones", "technology"],
    "Should churches remain tax-exempt?": ["churches", "tax exemption", "religion"],
    "Should corporal punishment be used in schools?": ["corporal punishment", "discipline", "education"],
    "Should education be free?": ["education", "tuition-free", "public education"],
    "Should election day be a national holiday?": ["election day", "voting rights", "national holiday"],
    "Should euthanasia or physician-assisted suicide be legal?": ["euthanasia", "physician-assisted suicide", "end-of-life care"],
    "Should everyone get a universal basic income?": ["universal basic income", "economic policy", "social welfare"],
    "Should felons who have completed their sentence be allowed to vote?": ["voting rights", "felony disenfranchisement", "criminal justice"],
    "Should fighting be allowed in hockey?": ["hockey", "sports", "athleticism"],
    "Should gay marriage be legal?": ["gay marriage", "marriage equality", "LGBTQ+ rights"],
    "Should government spending be reduced?": ["government spending", "fiscal policy", "budget cuts"],
    "Should hate speech be penalized more?": ["hate speech", "freedom of speech", "discrimination"],
    "Should holders of public offices resign on bad approval ratings?": ["public officials", "approval ratings", "political accountability"],
    "Should human cloning be banned?": ["human cloning", "biotechnology", "bioethics"],
    "Should humans attempt to contact extra-terrestrial life?": ["extraterrestrial life", "alien contact", "space exploration"],
    "Should insider trading be allowed?": ["insider trading", "stock market", "securities regulation"],
    "Should marijuana be a medical option?": ["medical marijuana", "cannabis", "drug policy"],
    "Should more gun control laws be enacted?": ["gun control", "firearms", "public safety"],
    "Should music that glorifies violence against women be banned?": ["music censorship", "violence against women", "media influence"],
    "Should net neutrality be restored?": ["net neutrality", "internet regulations", "equal access"],
    "Should nuclear weapons be abolished?": ["nuclear weapons", "international security", "disarmament"],
    "Should people become vegetarian?": ["vegetarianism", "animal rights", "health"],
    "Should performance-enhancing drugs be accepted in sports?": ["performance-enhancing drugs", "doping", "fair competition"],
    "Should prescription drugs be advertised directly to consumers?": ["direct-to-consumer advertising", "pharmaceutical industry", "healthcare"],
    "Should prostitution be legal?": ["prostitution", "sex work", "decriminalization"],
    "Should prostitution be legalized?": ["prostitution", "sex work", "legalization"],
    "Should recreational marijuana be legal?": ["recreational marijuana", "cannabis legalization", "drug policy"],
    "Should social networks be banned?": ["social media", "internet censorship", "free speech"],
    "Should social security be privatized?": ["social security", "retirement benefits", "privatization"],
    "Should student loan debt be easier to discharge in bankruptcy?": ["student loan debt", "bankruptcy", "financial aid"],
    "Should students have to wear school uniforms?": ["school uniforms", "dress code", "education"],
    "Should suicide be a criminal offense?": ["suicide", "mental health", "criminalization"],
    "Should teachers get tenure?": ["teacher tenure", "academic freedom", "education policy"],
    "Should the UNO become a world government?": ["UNO", "world government", "international relations"],
    "Should the US Electoral college be abolished?": ["Electoral college", "voting system", "elections"],
    "Should the death penalty be allowed?": ["death penalty", "capital punishment", "criminal justice"],
    "Should the federal minimum wage be increased?": ["minimum wage", "income inequality", "labor laws"],
    "Should the government allow illegal immigrants to become citizens?": ["illegal immigration", "citizenship", "immigration reform"],
    "Should the inheritance tax be raised?": ["inheritance tax", "estate tax", "taxation"],
    "Should the penny stay in circulation?": ["penny", "currency", "monetary policy"],
    "Should the press be subsidized?": ["press subsidies", "media industry", "journalism"],
    "Should the voting age be lowered?": ["voting age", "youth participation", "democracy"],
    "Should vigilantism be legal?": ["vigilantism", "citizen justice", "law enforcement"],
    "Should we imprison fewer people?": ["prison reform", "criminal justice system", "mass incarceration"],
    "Was the Iraq war legal?": ["Iraq war", "international law", "foreign policy"],
    "Are gender or racial quotas effective?": ["gender quotas", "racial quotas", "diversity"],
    "Can terrorism be justified?": ["terrorism", "justification", "violence"],
    "Do we need cash?": ["cash", "digital currency", "payment systems"],
    "Do we need sex education in schools?": ["sex education", "comprehensive education", "sexual health"],
    "How can we combat cyberbullying?": ["cyberbullying", "online harassment", "internet safety"],
    "Should agricultural subsidies be reduced?": ["agricultural subsidies", "farming industry", "government policy"],
    "Should all museums be free of charge?": ["museums", "cultural institutions", "accessibility"],
    "Should celibacy be abolished?": ["celibacy", "religious practice", "priesthood"],
    "Should endangered species be protected?": ["endangered species", "conservation", "biodiversity"],
    "Should stem cell research be expanded?": ["stem cell research", "medical research", "biotechnology"]
}
KEYWORDS_FROM_QUERY = {k: [word.lower() for word in v] for k, v in KEYWORDS_FROM_QUERY.items()}

ARGUMENTATIVE_WORDS_LIST= ['Therefore', 'In conclusion', 'Furthermore', 'Moreover', 'Consequently', 'As a result', 'However', 'Nevertheless', 'On the other hand', 'Despite this', 'Although', 'Nonetheless', 'In contrast', 'Whereas', 'Firstly', 'Secondly', 'Thirdly', 'Additionally', 'Another key point', 'Lastly', 'It is evident that', 'It is clear that', 'It is apparent that', 'It is undeniable that', 'This supports the idea that', 'This suggests that', 'One could argue that', 'It can be argued that', 'This illustrates that', 'This demonstrates that']
ARGUMENTATIVE_WORDS_LIST = [word.lower() for word in ARGUMENTATIVE_WORDS_LIST]

In [32]:
#helper functions
def get_avg_length_of_elements(text):
    total_length = 0
    for item in text:
        total_length += len(item)
    if len(text) == 0:
        return 0
    else:
        average_length = total_length / len(text)
        return average_length

In [33]:
#feature extractions
def extract_length_of_document(text):
    """Extracts the length of the document."""
    doc_length = len(text)
    return doc_length

def extract_length_of_paragraphs(text):
    """Extracts the length of paragraphs."""
    paragraphs = text.split("\n")
    paragraph_lengths = [nltk.sent_tokenize(paragraph) for paragraph in paragraphs]
    return get_avg_length_of_elements(paragraph_lengths)

def extract_length_of_sentences(text):
    """Extracts the length of sentences."""
    sentences = nltk.sent_tokenize(text)
    return get_avg_length_of_elements(sentences)

def extract_length_of_words(text):
    """Extracts the length of words."""
    words = nltk.word_tokenize(text)
    av_l = get_avg_length_of_elements(words)
    return av_l
 
def extract_number_of_stopwords(text):
    """Extracts the number of stopwords."""
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    num_stopwords = sum([1 for word in words if word.lower() in stop_words])
    return (num_stopwords / len(words))

def extract_number_of_special_characters(text):
    """Extracts the number of special characters."""
    special_chars = set(string.punctuation)
    num_special_chars = sum([1 for char in text if char in special_chars])
    return (num_special_chars / len(text))

def extract_number_of_numerics(text):
    """Extracts the number of numerics."""
    num_numerics = sum([1 for char in text if char.isdigit()])
    return (num_numerics / len(text))

def extract_number_of_uppercase_words(text):
    """Extracts the number of UPPERCASE WORDS and Uppercase Words."""
    words = nltk.word_tokenize(text)
    num_uppercase_words = sum([1 for word in words if word.isupper()])
    return (num_uppercase_words / len(words))

def extract_number_of_uppercase_sentence(text):
    """Extracts the number of UPPERCASE sentences. """
    sentences = nltk.sent_tokenize(text)
    num_uppercase_sentences = sum([1 for sen in sentences if sen[0].isupper()])
    return (num_uppercase_sentences / len(sentences))

def extract_number_of_academic_words(text):
    """Extracts the of academic quality words quality texts."""
    words = nltk.word_tokenize(text)
    academic_word_num = 0
    for word in words:
        w = Word(word)
        neutral_form = w.lemmatize()
        if neutral_form in academic_word_list:
            academic_word_num += 1
    return (academic_word_num / len(words))

def extract_number_of_profanity_words(text):
    """Extracts the of academic profanity words quality texts."""
    words = nltk.word_tokenize(text)
    profanity_word_num = 0
    for word in words:
        w = Word(word)
        neutral_form = w.lemmatize()
        if neutral_form in profanity_word_list:
            profanity_word_num += 1
    return (profanity_word_num / len(words))

def extract_vocabulary_richness(text):
    """Extracts the vocabulary richness with stopwords."""
    words = nltk.word_tokenize(text)
    l_words = []
    for word in words:
        w = Word(word)
        l_words.append(w.lemmatize())
    vocab_richness = len(set(l_words)) / len(words)
    return vocab_richness

def extract_vocabulary_richness_stopwords_removed(text):
    """Extracts the vocabulary richness without stopwords."""
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words_without_stopwords = [word for word in words if word.lower() not in stop_words]
    l_words = []
    for word in words_without_stopwords:
        w = Word(word)
        l_words.append(w.lemmatize())
    vocab_richness = len(set(l_words)) / len(words_without_stopwords)
    return vocab_richness

def extract_argument_ratio(text):
    """Extracts the argument ratio."""
    network = "tag-essays-dependency"
    url = "https://demo.webis.de/targer-api/targer-api/"
    headers = {"accept" : "application/json", "Content-Type": "text/plain"}
    r = requests.post(url+network, headers=headers, data=text.encode('utf-8'))
    number_arguments = count_arguments(r.text)
    words = nltk.word_tokenize(text)
    return number_arguments / len(words)

def extract_number_of_links(text):
    """Extracts the number of links."""
    words = nltk.word_tokenize(text)
    num_links = text.count("www.")
    return (num_links/len(words))

def extract_subjectivity(text):
    """Extracts the subjectivity of text per sentence."""
    sentences = nltk.sent_tokenize(text)
    subjectivity = 0
    sia = SentimentIntensityAnalyzer()
    for sentence in sentences:
        sentiment_scores = sia.polarity_scores(sentence)
        subjectivity += sentiment_scores['pos'] + sentiment_scores['neg']
    return (subjectivity / len(sentences))

def extract_topic_keyword(text, keyword:list):
    """Extracts the presence of a topic keyword in text."""
    tmp_text = text.lower()
    keyword_occurrence = sum((k in tmp_text for k in keyword)) / len(keyword)
    return keyword_occurrence

def extract_argumentative_words(text):
    """Extracts the presence of a topic keyword in text."""
    words = nltk.word_tokenize(text)
    tmp_text = text.lower()
    keyword_occurrence = sum((tmp_text.count(k) for k in ARGUMENTATIVE_WORDS_LIST)) / len(words)
    return keyword_occurrence

def extract_sentiment(text):
    """Extracts the sentiment of the text."""
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(text)
    compound_sentiment = sentiment_scores['compound']
    return compound_sentiment

def extract_readability(text):
    ##TODO edit?
    ##The library returns an error when there is less than 100 words (or more??) in the text
    ##In the following this is a workaround by appending the same text again. 
    ##If this does not work then use a dummy text of 'average readability (ChatGPT)'
    """Extracts the readability of the text using Flesch-Kincaid and SMOG indices."""
    dummy_text = "The importance of exercise cannot be overstated. Regular physical activity has numerous health benefits, including improved cardiovascular function, stronger muscles and bones, and decreased risk of chronic diseases such as diabetes and obesity. Additionally, exercise is a proven mood booster, helping to reduce feelings of anxiety and depression. Incorporating exercise into your daily routine doesn't have to be difficult or time-consuming. Simply taking a brisk walk or jog around the block, or doing a few sets of bodyweight exercises like push-ups and squats, can have a significant impact on your overall health and well-being. So, whether you prefer running, cycling, swimming, or any other form of physical activity, make sure to prioritize exercise as a crucial part of your daily routine."
    i=0
    while len(nltk.word_tokenize(text))<200:
            text = text + ' ' + text
            i+=1
            if(i>20):
                text = dummy_text
                break
    r = Readability(text)
    r_tups= [
        ["READABILITY_flesch_kincaid",r.flesch_kincaid().score],
        ["READABILITY_flesch",r.flesch().score],
        ["READABILITY_gunning_fog",r.gunning_fog().score],
        ["READABILITY_coleman_liau",r.coleman_liau().score],
        ["READABILITY_dale_chall",r.dale_chall().score],
        ["READABILITY_ari",r.ari().score],
        ["READABILITY_linsear_write",r.linsear_write().score],
        ["READABILITY_spache",r.spache().score]
    ]
    return  r_tups


def extract_ratio_of_sentence_types(text):
    """Extracts the ratio of different sentence types (questions, statements, etc.) in text."""
    sentences = nltk.sent_tokenize(text)
    question_count = 0
    statement_count = 0
    exclamation_count = 0
    for sentence in sentences:
        if sentence.endswith("?"):
            question_count += 1
        elif sentence.endswith("!"):
            exclamation_count += 1
        else:
            statement_count += 1
    total_sentences = len(sentences)
    question_ratio = question_count / total_sentences
    statement_ratio = statement_count / total_sentences
    exclamation_ratio = exclamation_count / total_sentences
    types = [
        ["SENTENCE_TYPE_question", question_ratio],
        ["SENTENCE_TYPE_statement", statement_ratio],
        ["SENTENCE_TYPE_exclamation", exclamation_ratio]
    ]
    return types 


def extract_number_of_named_entities(text):
    named_entities = []
    words = nltk.word_tokenize(text)
    tagged_words = nltk.pos_tag(words)
    chunked = ne_chunk(tagged_words)
    for chunk in chunked:
        if hasattr(chunk, 'label') and chunk.label() == 'NE':
            named_entities.append(chunk)
    return len(named_entities) / len(words)


def extract_number_of_spelling_mistakes(text):
    words = nltk.word_tokenize(text)
    num_spelling_mistakes = 0
    for word in words:
        if not enchant_dict.check(word): # Check if the word is misspelled
            num_spelling_mistakes += 1
    return num_spelling_mistakes/len(words)


### special functions for argument tagging

def count_arguments(text):
    claims = count_claims(text)
    premises = count_premises(text)
    return claims + premises

def count_claims(text):
    count = 0
    for item in json.loads(text):
        for element in item:
            if "C" in element['label'] and float(element['prob']) > 0.5:
                count += 1
    return count

def count_premises(text):
    count = 0
    for item in json.loads(text):
        for element in item:
            if "B" in element['label'] and float(element['prob']) > 0.5:
                count += 1
    return count

In [34]:
def extract_feature(text, feature_name, query):
    """Extracts the specified feature from the text."""
    feature_name = feature_name.upper()

    if feature_name == "LENGTH_OF_DOCUMENT":
        return len(text)
    elif feature_name == "LENGTH_OF_PARAGRAPHS":
        return extract_length_of_paragraphs(text)
    elif feature_name == "LENGTH_OF_SENTENCES":
        return extract_length_of_document(text)
    elif feature_name == "LENGTH_OF_WORDS":
        return extract_length_of_words(text)
    elif feature_name == "STOPWORD_NUM":
        return extract_number_of_stopwords(text)
    elif feature_name == "SPECCHARS_NUM":
        return extract_number_of_special_characters(text)
    elif feature_name == "NUMERICS_NUM":
        return extract_number_of_numerics(text)
    elif feature_name == "FULLUPPERCASEWORD_NUM":
        return extract_number_of_uppercase_words(text)
    elif feature_name == "UPPERCASEWORD_NUM":
        return extract_number_of_uppercase_words(text)
    elif feature_name == "UPPERCASESENTENCE_NUM":
        return extract_number_of_uppercase_sentence(text)

    elif feature_name == "VOCABULARY_RICHNESS":
        return extract_vocabulary_richness(text)
    elif feature_name == "VOCABULARY_RICHNESS_WITHOUT_STOPS":
        return extract_vocabulary_richness_stopwords_removed(text)
    elif feature_name == "ACADEMIC_WORDS":
        return extract_number_of_academic_words(text)
    elif feature_name == "PROFANITY_WORDS":
        return extract_number_of_profanity_words(text)
    elif feature_name == "ARGUMENTATIVE_WORDS":
        return extract_argumentative_words(text)
    elif feature_name == "ARGUMENT RATIO":
        return extract_argument_ratio(text)
    elif feature_name == "SUBJECTIVITY":
        return extract_subjectivity(text)
    elif feature_name == "LINKS_NUM":
        return extract_number_of_links(text)
    elif feature_name == "SENTIMENT":
        return extract_sentiment(text)
    elif feature_name == "NAMEDENTITIES_NUM":
        return extract_number_of_named_entities(text)
    elif feature_name == "SPELLINGMISTAKES_NUM":
        return extract_number_of_spelling_mistakes(text)
    elif feature_name == 'TOPIC_KEYWORDS':
        return extract_topic_keyword(text, keyword=KEYWORDS_FROM_QUERY[query])
    
    #these functions return a list of lists!
    elif feature_name.startswith("READABILITY"):
        return extract_readability(text)
    elif feature_name == "SENTENCE_TYPE":
        return extract_ratio_of_sentence_types(text)
    else:
        print("Error: Invalid feature name.")
        return -1

In [54]:
#main function
import random
import pandas as pd
from tqdm import tqdm

def annotate_features(dfold, feature_list):
    df = dfold.copy()
    for feature in tqdm(feature_list):
        print(feature)
        if feature != "READABILITY" and feature != "SENTENCE_TYPE": 
            df[feature] = -1
            for i, row in df.iterrows():
                df.at[i,feature] = extract_feature(row["clean_text"], feature, row['query'])
        elif feature == "READABILITY":
            df["READABILITY_flesch_kincaid"] = -1
            df["READABILITY_flesch"] = -1
            df["READABILITY_gunning_fog"] = -1
            df["READABILITY_coleman_liau"] = -1
            df["READABILITY_dale_chall"] = -1
            df["READABILITY_ari"] = -1
            df["READABILITY_linsear_write"] = -1
            df["READABILITY_spache"] = -1
            for i, row in df.iterrows():
                read_tupel = extract_feature(row["clean_text"], feature, row['query'])
                for t in read_tupel:
                    df.loc[i, t[0]] = t[1]
        elif feature == "SENTENCE_TYPE":
            df["SENTENCE_TYPE_statement"] = -1   
            df["SENTENCE_TYPE_question"] = -1   
            df["SENTENCE_TYPE_exclamation"] = -1
            for i, row in df.iterrows():
                read_tupel = extract_feature(row["clean_text"], feature, row['query'])
                for t in read_tupel:
                    df.loc[i, t[0]] = t[1]
    return df




#annotated_df = annotate_features(df, feature_list)
# saving as tsv file
#annotated_df.to_csv('data/qrels_args_docs_train_features.tsv', sep="\t")

In [ ]:
#import data
df = pd.read_csv("data/train-val-splits/qrels_args_docs_val.tsv", sep='\t')

df["clean_text"] = df["prem"].apply(clean_text).dropna()
df["query"] = df["qid"].apply(fetch_original_query)
df = df.dropna()

annotated_df = annotate_features(df, feature_list)
# saving as tsv file
annotated_df.to_csv("data/train-val-splits/qrels_args_docs_val_features_arg_ratio.tsv", sep="\t", index=False)

In [ ]:
#import data
df = pd.read_csv("data/train-val-splits/qrels_args_docs_train.tsv", sep='\t')

df["clean_text"] = df["prem"].apply(clean_text).dropna()
df["query"] = df["qid"].apply(fetch_original_query)
df = df.dropna()

annotated_df = annotate_features(df, feature_list)
# saving as tsv file
annotated_df.to_csv("data/train-val-splits/qrels_args_docs_train_features_arg_ratio.tsv", sep="\t", index=False)

In [ ]:
df = pd.read_csv("data/chatnoir_10_custom_stopw_lemmas.tsv", sep='\t')

df["clean_text"] = df["html_plain"].apply(clean_text).dropna()
df["query"] = df["qid"].apply(fetch_original_query)
df = df.dropna()

annotated_df = annotate_features(df, feature_list)
# saving as tsv file
annotated_df.to_csv("data/qrels_args_docs_test_features_arg_ratio.tsv", sep="\t", index=False)

In [191]:
import os

dfs = []

for file in os.listdir('data'):
    if "ArgumentRatio" in file and "_val_" in file:
        dfs.append(pd.read_csv(os.path.join('data',file), sep='\t'))
        
df_comb = pd.concat(dfs)
df_comb = df_comb[['doc_id', 'q_id', 'ARGUMENT RATIO']]
len(df_comb)

692

In [132]:
#df_comb.columns
#df_comb["hash"] = str(df_comb.q_id)+df_comb.doc_id

In [192]:
df_read = pd.read_csv('data/qrels_args_docs_val_features.tsv', sep="\t")
len(df_read)
#df_read['hash'] = str(df_read.q_id)+df_read.doc_id

#list(df_comb.hash) == list(df_read.hash)

692

In [145]:
#df_comb[df_comb.q_id == 94]

In [153]:
#for idx, row in df_read.iterrows():
#    df1 = df_comb[df_comb.q_id == row.q_id]
#    df2 = df1[df1.doc_id == row.doc_id]
#    if len(df2) > 1:
#        print(df2)
#        print()

In [194]:
merged_df = pd.merge(df_read, df_comb, on=['q_id', 'doc_id'], how='right').drop_duplicates()
len(merged_df)

691

In [195]:
merged_df.to_csv('data/qrels_args_docs_val_features_arg_ratio.tsv', sep="\t", index=False)

In [178]:
import os

dfs = []

for file in os.listdir('data'):
    if "ArgumentRatio" in file and "_chatnoir_" in file:
        dfs.append(pd.read_csv(os.path.join('data',file), sep='\t'))
        
df_comb = pd.concat(dfs)
df_comb = df_comb[['docno', 'qid', 'ARGUMENT RATIO']]
len(df_comb)

500

In [179]:
df_read = pd.read_csv('data/qrels_args_docs_test_features.tsv', sep="\t")
len(df_read)
#df_read['hash'] = str(df_read.q_id)+df_read.doc_id

#list(df_comb.hash) == list(df_read.hash)

500

In [180]:
merged_df = pd.merge(df_read, df_comb, on=['qid', 'docno'], how='right').drop_duplicates()
len(merged_df)

500

In [181]:
merged_df.to_csv('data/qrels_args_docs_test_features_arg_ratio.tsv', sep="\t", index=False)